In [1]:
# we start by importing some libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


### let's export the content of our working directory
import os
print(os.listdir("../input/malware-revealer-dataset-json-v1/malware-revealer-dataset-json-v1"))


['1', '0']


we can see that we have 2 dirctories, dir 1 is the one that containes the malwares and the 0 for the bening files 

# import the features from  files

In [2]:
## the full path to our PWD
dataset_dir = "../input/malware-revealer-dataset-json-v1/malware-revealer-dataset-json-v1"

### malware_jsons and benign_jsons are two variables that hold the names of the diffrent files in the two classes 
malware_jsons = os.listdir(dataset_dir+"/1")
benign_jsons = os.listdir(dataset_dir+"/0")

In [3]:
print("number of malware fils : ",len(malware_jsons))
print("number of benign files : ",len(benign_jsons))

number of malware fils :  2165
number of benign files :  2495


now we import all the features of the files, and we save every file's name with our dataset.

In [4]:
import json

malware_array = []
for file in malware_jsons:
    with open(dataset_dir+"/1/"+file) as f:
        malware_array.append(json.load(f))
        malware_array[-1]['file_name'] = file[:len(file)-5]

benign_array = []
for file in benign_jsons:
    with open(dataset_dir+"/0/"+file) as f:
        benign_array.append(json.load(f))
        benign_array[-1]['file_name'] = file[:len(file)-5]

Now let's try to print some simples

In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [6]:
pp.pprint(malware_array[8])

{   'MZ': 1,
    'PE_signature': [80, 69, 0, 0],
    'avg_length': 6.826234182855106,
    'byte_count': [   227958,
                      3954,
                      1306,
                      2309,
                      2459,
                      2079,
                      1377,
                      1384,
                      773,
                      1338,
                      525,
                      541,
                      747,
                      541,
                      503,
                      568,
                      765,
                      551,
                      1180,
                      437,
                      634,
                      496,
                      477,
                      463,
                      592,
                      428,
                      377,
                      433,
                      569,
                      399,
                      365,
                      411,
                      742,
           

In [7]:
pp.pprint(benign_array[0])

{   'MZ': 1,
    'PE_signature': [80, 69, 0, 0],
    'avg_length': 10.232747310417214,
    'byte_count': [   482456,
                      28157,
                      14897,
                      17179,
                      23067,
                      9882,
                      10243,
                      8383,
                      29211,
                      5984,
                      8508,
                      5807,
                      19254,
                      5963,
                      5498,
                      15173,
                      17199,
                      7256,
                      4003,
                      3348,
                      9262,
                      11041,
                      3912,
                      3994,
                      10204,
                      7137,
                      4393,
                      4937,
                      7789,
                      7389,
                      4428,
                      5083,
    

like we see there are a lot of features in the 2 kind of file, from code,file and virtual size to file's name, and sections of code, adding to that some imported  and exported functions, of course we have to clean all this chaos, we starte by the obvois features.<br>
### firstly
we create a pandas dataframe with some obvois columns and we start to fill it with the features in the files (the are all in the benign_array and malware_array)


In [8]:
COLUMNS = ["target","file_name","avg_length","byte_count_mean","byte_count_median","has_characteristics","has_debug","has_resources","has_signature","has_tls",\
                                 "section_counts","has_symbols","paths_count","registry_count","has_weird_sections","file_size","sizeof_code","virtual_size","url_counts"]
database = pd.DataFrame(columns = COLUMNS)

all the columns or the features that i add in the above code don't need any encoding. so now we will try to extract all the data and fill the database <br>
but before we start we have to find a solution for the features that needs encoding and in our case we will use the one hot encoding method, these features are 
* DLL_charactiristics
* SUBSYSTEM
* imported_functions
* dll_libs

### DLL_charactiristics <br>
in the dll_charactiristics we should first see how much do we have, and what are the diffrent type of dll_charactiristics do we have, if we have a small number we can use them all in our dataset <br>
in order to see how much types do we have, we define a fucntion that counts the number of each dll_char and list them 

In [9]:
from collections import Counter

def get_Counter(field,single_value = False):
    malware_counter = Counter()
    benign_counter =  Counter()

    for file in malware_array:
        if field in file:
            if not single_value :
                malware_counter += Counter(file[field])
            else:
                malware_counter += Counter([file[field]])


    for file in benign_array:
        if field in file:
            if not single_value :
                malware_counter += Counter(file[field])
            else:
                malware_counter += Counter([file[field]])
    return malware_counter, benign_counter

In [10]:
malware,benign = get_Counter('dll_characteristics')
malware += benign
dll_char = list(dict(malware).keys())
dll_char

['DYNAMIC_BASE',
 'NX_COMPAT',
 'TERMINAL_SERVER_AWARE',
 'NO_SEH',
 'NO_BIND',
 'NO_ISOLATION',
 'GUARD_CF',
 'HIGH_ENTROPY_VA',
 'FORCE_INTEGRITY']

as we see, we have only 9 kinds, so we can one hot encode them

## SUBSYSTEM <br> 
now let's see how much diffrent subsystem do we have

In [11]:
malware,benign = get_Counter('subsystem',True)
malware += benign
subsystem = list(dict(malware).keys())
subsystem = subsystem[:-1]
subsystem

['SUBSYSTEM.WINDOWS_GUI',
 'SUBSYSTEM.WINDOWS_CUI',
 'SUBSYSTEM.NATIVE',
 'SUBSYSTEM.WINDOWS_BOOT_APPLICATION']

the same thing for the subsystem features. we encode it by the one hot endocing 

## imported functions

In [12]:
malware,benign = get_Counter('imported_functions')
malware += benign
list(dict(malware).keys())

['WritePrivateProfileStringA',
 'TlsSetValue',
 'DeleteCriticalSection',
 'RegQueryValueExA',
 'WNetCancelConnectionA',
 'SysFreeString',
 'ShellExecuteA',
 'URLDownloadToFileA',
 'GetKeyboardType',
 'InternetGetConnectedState',
 'WSACleanup',
 'EVENT_SINK_GetIDsOfNames',
 '__vbaVarTstGt',
 '__vbaVarSub',
 '__vbaStrI2',
 '_CIcos',
 '_adj_fptan',
 '__vbaVarMove',
 '__vbaStrI4',
 '__vbaVarVargNofree',
 '__vbaFreeVar',
 '__vbaStrVarMove',
 '__vbaLenBstr',
 '__vbaLateIdCall',
 '__vbaFreeVarList',
 '__vbaEnd',
 '_adj_fdiv_m64',
 'EVENT_SINK_Invoke',
 '__vbaFreeObjList',
 '_adj_fprem1',
 '__vbaVarCmpNe',
 '__vbaStrCat',
 '__vbaSetSystemError',
 '__vbaHresultCheckObj',
 '__vbaLenVar',
 '_adj_fdiv_m32',
 'Zombie_GetTypeInfo',
 '__vbaForEachCollObj',
 '__vbaVarForInit',
 '__vbaExitProc',
 '__vbaObjSet',
 '__vbaOnError',
 '_adj_fdiv_m16i',
 '__vbaObjSetAddref',
 '_adj_fdivr_m16i',
 '__vbaBoolVar',
 '__vbaFpR8',
 '__vbaVarTstLt',
 '__vbaBoolVarNull',
 '_CIsin',
 '__vbaNextEachCollObj',
 '__vbaChk

in the imported functions it's deffrent so we will try to take the 10 most common in every kind of files and one hot encoding it 

In [13]:
malware,benign = get_Counter('imported_functions')
imported_functions = list(set(dict(malware.most_common(10)).keys()) | set(dict(benign.most_common(10)).keys()))
imported_functions

['SetUnhandledExceptionFilter',
 'UnhandledExceptionFilter',
 'GetProcAddress',
 'GetCurrentProcessId',
 'GetCurrentProcess',
 'GetLastError',
 'Sleep',
 'TerminateProcess',
 'CloseHandle',
 'GetCurrentThreadId']

no we have a small number of imported fucntions, we will use these functions in our dataset

## dll_libs <br>
it's the same as the imported functions we will take 10 most common in each type of files

In [14]:
malware,benign = get_Counter('libs')
libs = list(set(dict(malware.most_common(10)).keys()) | set(dict(benign.most_common(10)).keys()))
libs

['kernel32.dll',
 'ntdll.dll',
 'KERNEL32.dll',
 'ole32.dll',
 'OLEAUT32.dll',
 'GDI32.dll',
 'ADVAPI32.dll',
 'SHELL32.dll',
 'msvcrt.dll',
 'USER32.dll']

In [15]:
def fill_database(target, data, dataframe,columns):
    # we save the files that we couldn't extract (we can't find the features that we should find), so the json file is not a good one
    bad_files = list()
    not_defult_columns = ["target","byte_count_median","byte_count_mean","has_characteristics","has_symbols","has_weird_sections"]
    for file in data:
        try:
            row = { key: file[key] for key in columns if key not in not_defult_columns }
            byte_count = np.array(file['byte_count'])
            row['byte_count_mean'] = byte_count.mean()
            row['byte_count_median'] = np.median(byte_count)
            row['target'] = target
            row['has_characteristics'] = int(len(file['characteristics']) > 0)
            row['has_symbols'] = int(file['symbols'] > 0)
            ## we know that all normal sections in the pe files should start with "." like ".data" or ".rdata" so we look at the names of the sections in the files once we find 
            # a section with a name that not start with . we set the "has_weird_sections" to 1 to say that there is a not normal sections
            row['has_weird_sections'] = int(len([1 for i in list(file['sections'].keys()) if not i.startswith(".")]) > 0)
            ### the one hot encodeing part 
            ### DLL_charactiristics
            ### we have a limited number of dll_char
            
            row = one_hot_encoding(row, dll_char, file['dll_characteristics'],prefix='dll_char')
            
            ### for the SUBSYSTEM it's the same thing we have 4 diffrent types her
           
            row  = one_hot_encoding(row,subsystem,[file['subsystem']],prefix='subsys_')
           
            ### for the imported fucntions 
            row  = one_hot_encoding(row,imported_functions,file['imported_functions'],prefix='imp_func')
            
            ## for the dll libs
            row  = one_hot_encoding(row,libs,file['libs'],prefix='dll_libs')
            
            dataframe = dataframe.append(row,ignore_index = True)
        except:
            bad_files.append(file)
    return dataframe, bad_files

In [16]:
def one_hot_encoding(row,columns,data,prefix):
    for feature in columns:
        if feature in data:
            row[prefix+'_'+feature] = 1
        else:
            row[prefix+'_'+feature] = 0
    return row

In [17]:
database,bad_files = fill_database(1,malware_array,database,COLUMNS)

In [18]:
print(database.shape)
print(len(bad_files))

(2164, 52)
1


we have just one bad file which is not bad at all, 1 bad from 2165 good files is totaly okay

In [19]:
database,bad_files = fill_database(0,benign_array,database,COLUMNS)

In [20]:
print(database.shape)
print(len(bad_files))

(4646, 52)
13


In [21]:
database.head()

,target,file_name,avg_length,byte_count_mean,byte_count_median,has_characteristics,has_debug,has_resources,has_signature,has_tls,section_counts,has_symbols,paths_count,registry_count,has_weird_sections,file_size,sizeof_code,virtual_size,url_counts,dll_char_DYNAMIC_BASE,dll_char_FORCE_INTEGRITY,dll_char_GUARD_CF,dll_char_HIGH_ENTROPY_VA,dll_char_NO_BIND,dll_char_NO_ISOLATION,dll_char_NO_SEH,dll_char_NX_COMPAT,dll_char_TERMINAL_SERVER_AWARE,dll_libs_ADVAPI32.dll,dll_libs_GDI32.dll,dll_libs_KERNEL32.dll,dll_libs_OLEAUT32.dll,dll_libs_SHELL32.dll,dll_libs_USER32.dll,dll_libs_kernel32.dll,dll_libs_msvcrt.dll,dll_libs_ntdll.dll,dll_libs_ole32.dll,imp_func_CloseHandle,imp_func_GetCurrentProcess,imp_func_GetCurrentProcessId,imp_func_GetCurrentThreadId,imp_func_GetLastError,imp_func_GetProcAddress,imp_func_SetUnhandledExceptionFilter,imp_func_Sleep,imp_func_TerminateProcess,imp_func_UnhandledExceptionFilter,subsys__SUBSYSTEM.NATIVE,subsys__SUBSYSTEM.WINDOWS_BOOT_APPLICATION,subsys__SUBSYSTEM.WINDOWS_CUI,subsys__SUBSYSTEM.WINDOWS_GUI
0,1,0e0f721a297d67ec9eb501ef6a270da922b7be9ae6f657...,8.889809,2150.269531,1734.5,1,0,1,0,1,4,0,2,0,1,550469,24576,98304,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,021b2618be9a4a8b301d65c6174f37ceff53c19107e699...,9.203319,4197.625000,1854.5,1,0,1,0,0,3,0,24,0,0,1074592,606208,1122304,51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,0f7b80f1b34b7fe0a83c843449b4344b7ad2ce4aacc58c...,12.480438,4416.000000,2020.5,1,0,1,0,0,4,0,4,0,0,1130496,647168,1384448,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1,0b5d94c653f73e79da239475d8ad5d584bceb065058b51...,9.887520,3988.167969,1560.0,1,0,1,0,0,5,0,7,0,0,1020971,20480,53248,68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,0a62f64ba9ed99fd175adc0b6d8407d029e65a979ae04b...,9.857459,2438.687500,1060.0,1,1,1,1,0,4,0,4,0,0,624304,5120,24576,55,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0


In [22]:
database.to_csv("database")